In [1]:
%load_ext autoreload
%autoreload 2
%cd /anhvth5/vision-projects/DiffSynth-Studio

/anhvth5/vision-projects/DiffSynth-Studio


In [2]:
from examples.ExVideo.ExVideo_svd_train_custom import *
from diffsynth.models.svd_unet import SVDUNet, PushBlock, PopBlock, PopMixBlock
from avcv.all import *
from fastcore.all import patch
args = parse_args().parse_known_args()[0]

In [3]:
args.num_frames = 8
args.steps_per_epoch = 500
args.pretrained_path =  "models/stable_video_diffusion/svd_xt.safetensors"
args.height = 512
args.width = 288
args.learning_rate = 1e-5

ds_path = './datasets/tiktokdance/'
steps_per_epoch = 500
#-----
dataset = TextVideoDataset(
        ds_path,
        os.path.join(ds_path, "metadata.json"),
        training_shapes=[(args.num_frames, 1, args.num_frames, 512, 384)],
        steps_per_epoch=args.steps_per_epoch,
    )

train_loader = torch.utils.data.DataLoader(
    dataset,
    shuffle=True,
    batch_size=1,
    num_workers=2,
)

In [26]:
%%time
if not 'model' in dir():
    model = LightningModel(
        learning_rate=args.learning_rate,
        svd_ckpt_path=args.pretrained_path,
        add_positional_conv=args.num_frames,
        contrast_enhance_scale=args.contrast_enhance_scale
    )
model.cuda();

CPU times: user 684 ms, sys: 719 ms, total: 1.4 s
Wall time: 1.4 s


In [22]:
batch = next(iter(train_loader))

In [31]:
frames = batch["frames_0"][0]
control_frames = batch['control_frames_0'][0]
frames.shape, control_frames.shape

(torch.Size([3, 8, 512, 384]), torch.Size([3, 8, 512, 384]))

In [ ]:
with torch.no_grad(), torch.cuda.amp.autocast():
    loss, reweighted_loss = model.calculate_loss(frames.cuda(), control_frames.cuda())

> /anhvth5/vision-projects/DiffSynth-Studio/examples/ExVideo/ExVideo_svd_train_custom.py(372)calculate_loss()
    371             import ipdb; ipdb.set_trace()
--> 372             control_latents = self.encode_video_with_vae(control_frames) / self.vae_encoder.scaling_factor
    373 



ipdb>  control_frames.shape


torch.Size([3, 8, 512, 384])


In [33]:
loss

tensor(0.5097, device='cuda:0')

In [9]:
frames = batch["frames_0"][0]
control_frames = batch['control_frames_0'][0]

In [10]:
frames.shape, control_frames.shape

(torch.Size([3, 8, 512, 384]), torch.Size([1, 1, 3, 512, 384]))

In [ ]:
with torch.cuda.amp.autocast():
    loss = model.calculate_loss(frames.cuda().half())

In [ ]:
loss